In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Preprocessing

In [ ]:
# Import data
df = pd.read_excel('/content/drive/Shareddrives/ENGS Final Project/Data/Credit Ratings/Company Screening Report.xlsx', sheet_name = "Screening") # Import the dataset we will be analyzing
print(df.shape)
df.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/packaging/custom.py:212: UserWarning: Unknown type for Created By
  warn(f"Unknown type for {prop.name}")


(1368, 16)


,Company Name,Exchange:Ticker,S&P Entity Credit Rating Last Review Date - Issuer Credit Rating - Foreign Currency LT [Latest] (Last Review Date),S&P Entity Credit Rating - Issuer Credit Rating - Foreign Currency LT [Latest] (Rating),S&P Entity Credit Rating Date - Issuer Credit Rating - Foreign Currency LT [Latest] (Rating Date),Exchanges [Primary Listing],Exchanges [Secondary Listings],"Market Capitalization [My Setting] [Latest] ($USDmm, Historical rate)",EBITDA / Interest Exp. [LTM],Total Liabilities/Total Assets % [Latest Quarter],"Common Dividends Paid [LTM] ($USDmm, Historical rate)",Total Debt to Capital [LTM],"Total Assets [Latest Annual] ($USDmm, Historical rate)","Total Debt [Latest Annual] ($USDmm, Historical rate)","Total Equity [Latest Annual] ($USDmm, Historical rate)",EBITDA Margin % [LTM]
0,"SK Telecom Co., Ltd. (KOSE:A017670)",KOSE:A017670,2023-02-23,A-,2012-02-14,-,"NYSE:SKM\nCompany: SK Telecom Co., Ltd.\n(IQT2...",8394.4,12.5,59.4,0,36.2,23226.5,5723.4,9430,28.9
1,Twilio Inc. (NYSE:TWLO),NYSE:TWLO,2023-02-24,BB,2021-03-03,New York Stock Exchange (NYSE),-,10183.7,-,16.2,0,10.7,11609.7,1168.8,9732.6,-1.64
2,Belden Inc. (NYSE:BDC),NYSE:BDC,2023-02-24,BB,2011-03-18,New York Stock Exchange (NYSE),-,3338.2,13.1,64,-8.5,51.8,3240.2,1304.2,1166.3,16.6
3,Hudbay Minerals Inc. (TSX:HBM),TSX:HBM,2023-02-24,B,2019-10-18,-,NYSE:HBM\nCompany: Hudbay Minerals Inc.\n(IQT1...,1877.3,7.68,59.1,-4.46,36.8,4325.9,1245.2,1571.8,39.8
4,NiSource Inc. (NYSE:NI),NYSE:NI,2023-02-25,BBB+,2015-06-18,New York Stock Exchange (NYSE),NYSE:NI.PRB\nCompany: NiSource Inc.\n(IQT59328...,10852.2,-,-,-,-,-,-,-,-


In [ ]:
# Convert columns to numerical
columns_to_convert = [
    'Market Capitalization [My Setting] [Latest] ($USDmm, Historical rate)',
    'EBITDA / Interest Exp. [LTM]',
    'Total Liabilities/Total Assets % [Latest Quarter]',
    'Common Dividends Paid [LTM] ($USDmm, Historical rate)',
    'Total Debt to Capital [LTM]',
    'Total Assets [Latest Annual] ($USDmm, Historical rate)',
    'Total Debt [Latest Annual] ($USDmm, Historical rate)',
    'Total Equity [Latest Annual] ($USDmm, Historical rate)',
    'EBITDA Margin % [LTM]'
]
df[columns_to_convert] = df[columns_to_convert].replace('-', np.nan).apply(pd.to_numeric, errors='coerce')

# Rename variables for ease of use
df.rename(columns={
    'Company Name':'company',
    'Market Capitalization [My Setting] [Latest] ($USDmm, Historical rate)': 'market_cap',
    'EBITDA / Interest Exp. [LTM]': 'ebitda_interest_exp',
    'Total Assets [Latest Annual] ($USDmm, Historical rate)': 'total_assets',
    'Total Debt [Latest Annual] ($USDmm, Historical rate)': 'total_debt',
    'Total Equity [Latest Annual] ($USDmm, Historical rate)': 'total_equity',
    'EBITDA Margin % [LTM]': 'ebitda_margin',
    'S&P Entity Credit Rating - Issuer Credit Rating - Foreign Currency LT [Latest] (Rating)': 'latest_credit_rating',
    'Common Dividends Paid [LTM] ($USDmm, Historical rate)': 'dividends_paid'
}, inplace=True)

df.dtypes

company                                                                                                                       object
Exchange:Ticker                                                                                                               object
S&P Entity Credit Rating Last Review Date - Issuer Credit Rating - Foreign Currency LT [Latest] (Last Review Date)    datetime64[ns]
latest_credit_rating                                                                                                          object
S&P Entity Credit Rating Date - Issuer Credit Rating - Foreign Currency LT [Latest] (Rating Date)                     datetime64[ns]
Exchanges [Primary Listing]                                                                                                   object
Exchanges [Secondary Listings]                                                                                                object
market_cap                                                           

In [ ]:
# Create new variables for key metrics
df['dividends'] = (df['dividends_paid'] != 0).astype(int)
df['market_to_book'] = (df['total_assets'] + df['market_cap'] - df['total_equity']) / df['total_assets']
df['debt_to_assets'] = df['total_debt'] / df['total_assets']


# Filter for specific variables
df = df[['company', 'market_cap', 'ebitda_interest_exp', 'debt_to_assets', 'dividends', 'ebitda_margin', 'market_to_book', 'latest_credit_rating']]

# Drop rows with missing data
df.replace('-', np.nan, inplace=True)  # Ensure all '-' have been replaced by np.nan
df.dropna(inplace=True)

print(df.shape)


# Group ratings below CCC+ into the same catgegory

def group_ratings(rating):
    # Define the order of ratings from highest to lowest
    rating_order = ['AAA', 'AA+', 'AA', 'AA-', 'A+', 'A', 'A-', 'BBB+', 'BBB', 'BBB-', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'CCC+', 'CCC', 'CCC-', 'CC', 'C', 'D']
    # If the rating is not in the list or is CCC or lower, classify it as 'CCC or lower'
    if rating not in rating_order or rating_order.index(rating) >= rating_order.index('CCC+'):
        return 'CCC+ or lower'
    else:
        return rating

# Apply the function to the 'latest_credit_rating' column
df['latest_credit_rating_adj'] = df['latest_credit_rating'].apply(group_ratings)

df.head()

(1120, 8)


<ipython-input-12-e65da3962a61>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace('-', np.nan, inplace=True)  # Ensure all '-' have been replaced by np.nan
<ipython-input-12-e65da3962a61>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-12-e65da3962a61>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latest_credit_rating_ad

,company,market_cap,ebitda_interest_exp,debt_to_assets,dividends,ebitda_margin,market_to_book,latest_credit_rating,latest_credit_rating_adj
0,"SK Telecom Co., Ltd. (KOSE:A017670)",8394.4,12.50,0.246417,0,28.9,0.955413,A-,A-
2,Belden Inc. (NYSE:BDC),3338.2,13.10,0.402506,1,16.6,1.670298,BB,BB
3,Hudbay Minerals Inc. (TSX:HBM),1877.3,7.68,0.287848,1,39.8,1.070621,B,B
5,Eni S.p.A. (BIT:ENI),49475.3,2.25,0.238776,1,19.3,0.937912,A-,A-
6,"Fiserv, Inc. (NYSE:FI)",88611.4,7.43,0.254351,0,38.0,1.637500,BBB,BBB


In [ ]:
df['latest_credit_rating_adj'].value_counts().sort_index()

A                 37
A+                17
A-                70
AA                 4
AA+                1
AA-               14
AAA                2
B                 64
B+               112
B-                38
BB               115
BB+              108
BB-              100
BBB              167
BBB+             122
BBB-             110
CCC+ or lower     39
Name: latest_credit_rating_adj, dtype: int64

## Modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, r2_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Assuming your DataFrame is named df and is already preprocessed as per previous instructions

# Encode the 'latest_credit_rating' column since Random Forest cannot handle categorical variables directly
le = LabelEncoder()
df['latest_credit_rating_encoded'] = le.fit_transform(df['latest_credit_rating_adj'])

# Define the feature variables (X) and the target variable (y)
X = df[['market_cap', 'ebitda_interest_exp', 'debt_to_assets', 'dividends', 'ebitda_margin', 'market_to_book']]
y = df['latest_credit_rating_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
model = RandomForestClassifier(random_state=42)

# Train the classifier
model.fit(X_train, y_train)

# Predict the credit ratings on the test set
y_pred = model.predict(X_test)

# Evaluate the model
unique_labels = np.unique(np.concatenate((y_test, y_pred)))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"R^2: {r2_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, labels=unique_labels, target_names=le.inverse_transform(unique_labels)))


Accuracy: 0.25
R^2: -0.3715851367225633
               precision    recall  f1-score   support

            A       0.00      0.00      0.00        11
           A+       0.50      0.20      0.29         5
           A-       0.08      0.07      0.08        14
          AA-       0.00      0.00      0.00         2
          AAA       0.00      0.00      0.00         1
            B       0.30      0.25      0.27        12
           B+       0.32      0.50      0.39        20
           B-       0.25      0.22      0.24         9
           BB       0.17      0.25      0.20        20
          BB+       0.25      0.20      0.22        20
          BB-       0.42      0.17      0.24        29
          BBB       0.27      0.46      0.34        28
         BBB+       0.38      0.29      0.33        28
         BBB-       0.16      0.21      0.18        19
CCC+ or lower       0.00      0.00      0.00         6

     accuracy                           0.25       224
    macro avg       0.2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Logistic regression
from sklearn.linear_model import LogisticRegression

# Initialize the Random Forest classifier
model = LogisticRegression(random_state = 42)

# Train the classifier
model.fit(X_train, y_train)

# Predict the credit ratings on the test set
y_pred = model.predict(X_test)

# Evaluate the model
unique_labels = np.unique(np.concatenate((y_test, y_pred)))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"R^2: {r2_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, labels=unique_labels, target_names=le.inverse_transform(unique_labels)))

Accuracy: 0.125
R^2: -0.42750967444569987
               precision    recall  f1-score   support

            A       0.00      0.00      0.00        11
           A+       0.00      0.00      0.00         5
           A-       0.00      0.00      0.00        14
          AA-       0.00      0.00      0.00         2
          AAA       0.00      0.00      0.00         1
            B       0.00      0.00      0.00        12
           B+       0.00      0.00      0.00        20
           B-       0.00      0.00      0.00         9
           BB       0.00      0.00      0.00        20
          BB+       0.00      0.00      0.00        20
          BB-       0.00      0.00      0.00        29
          BBB       0.12      1.00      0.22        28
         BBB+       0.00      0.00      0.00        28
         BBB-       0.00      0.00      0.00        19
CCC+ or lower       0.00      0.00      0.00         6

     accuracy                           0.12       224
    macro avg       0

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wit